In [4]:
data = spark.read.csv("file:/home/hduser/hive/data/txns", header=False, inferSchema=True)\
.toDF("txnid","txndate","custid","sales_amount","category","product","city","state","paymenttype")


In [7]:
# Convert sales_date to timestamp
from pyspark.sql.functions import year, month, dayofmonth, dayofweek
from pyspark.sql.functions import unix_timestamp, col

df = data.withColumn("sales_date", unix_timestamp(col("txndate"), "MM-dd-yyyy").cast("timestamp"))

# Extract components from sales_date
df = df.withColumn("year", year(df["sales_date"]))
df = df.withColumn("month", month(df["sales_date"]))
df = df.withColumn("dayofmonth", dayofmonth(df["sales_date"]))
df = df.withColumn("dayofweek", dayofweek(df["sales_date"]))
#df.printSchema()

In [9]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
# Index categorical columns
indexers = [
    StringIndexer(inputCol="city", outputCol="cityIndex"),
    StringIndexer(inputCol="state", outputCol="stateIndex"),
    StringIndexer(inputCol="product", outputCol="productIndex"),
    StringIndexer(inputCol="category", outputCol="categoryIndex")
]

# One-hot encode the indexed columns
encoders = [
    OneHotEncoder(inputCol="cityIndex", outputCol="cityVec"),
    OneHotEncoder(inputCol="stateIndex", outputCol="stateVec"),
    OneHotEncoder(inputCol="productIndex", outputCol="productVec"),
    OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec"),
    OneHotEncoder(inputCol="year", outputCol="yearVec"),
    OneHotEncoder(inputCol="month", outputCol="monthVec"),
    OneHotEncoder(inputCol="dayofmonth", outputCol="dayofmonthVec"),
    OneHotEncoder(inputCol="dayofweek", outputCol="dayofweekVec")
]

In [10]:
# Assemble features
assembler = VectorAssembler(
    inputCols=[
        "cityVec", "stateVec", "productVec", "categoryVec",
        "yearVec", "monthVec", "dayofmonthVec", "dayofweekVec"
    ],
    outputCol="features"
)

In [12]:
# Define the model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="sales_amount", featuresCol="features")

In [14]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])

In [15]:
# Split the data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=12345)

# Train the model
model = pipeline.fit(train_data)

In [17]:
# Make predictions
predictions = model.transform(test_data)
predictions.select("sales_amount","prediction").show()

+------------+------------------+
|sales_amount|        prediction|
+------------+------------------+
|       98.81|103.43992292182297|
|       96.01| 94.05218561898673|
|       10.44|106.63102722957412|
|      137.64| 98.89382727700085|
|       35.56|105.20621383778517|
|       75.55| 101.8385665428838|
|      185.26|106.73035291809343|
|       165.1|103.25221717216539|
|       38.52|100.52372280919198|
|       32.65| 102.4858917479779|
|      176.63|103.56003263485428|
|       89.14|102.65884692454837|
|      150.82|106.89835636889265|
|      148.31| 99.86390592097561|
|      170.05|106.78091335144775|
|        39.8|102.52051053293334|
|      174.82| 102.5670180096427|
|       55.35| 98.76826173866877|
|      193.11|103.03461597905624|
|      119.77| 97.65855660875636|
+------------+------------------+
only showing top 20 rows



In [23]:
model.save("file:/home/hduser/SparkML-Workouts/sales_model")
print("Model Saved")

Model Saved


In [42]:
from pyspark.sql.functions import unix_timestamp, col,lit
data1 = spark.read.csv("file:/home/hduser/future-sales-data.csv", inferSchema = True)\
.toDF("txnid","txndate","custid","category","product","city","state","paymenttype")

df = data1.withColumn("sales_date", unix_timestamp(col("txndate"), "MM-dd-yyyy").cast("timestamp"))

# Extract components from sales_date
df = df.withColumn("year", year(df["sales_date"]))
df = df.withColumn("month", month(df["sales_date"]))
df = df.withColumn("dayofmonth", dayofmonth(df["sales_date"]))
df = df.withColumn("dayofweek", dayofweek(df["sales_date"]))
df = test_data.select("year","month","dayofmonth","dayofweek","state","city","product","category")
test_data.printSchema()

root
 |-- txnid: integer (nullable = true)
 |-- txndate: string (nullable = true)
 |-- custid: integer (nullable = true)
 |-- sales_amount: double (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- paymenttype: string (nullable = true)
 |-- sales_date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)



In [43]:
# Make predictions
predictions = model.transform(df)
predictions.select("year","month","dayofmonth","dayofweek","state","city","product","category","prediction").show()

+----+-----+----------+---------+--------------+-----------+--------------------+--------------------+------------------+
|year|month|dayofmonth|dayofweek|         state|       city|             product|            category|        prediction|
+----+-----+----------+---------+--------------+-----------+--------------------+--------------------+------------------+
|2011|   12|        17|        7|     Tennessee|Nashville  |        Field Hockey|         Team Sports|103.43992292182297|
|2011|    7|        14|        5|          Ohio|   Columbus|           Sandboxes|Outdoor Play Equi...| 94.05218561898673|
|2011|    1|        17|        2|          Iowa| Des Moines|        Snowmobiling|       Winter Sports|106.63102722957412|
|2011|   10|        20|        5|        Hawaii| Honolulu  |             Fencing|       Combat Sports| 98.89382727700085|
|2011|    5|        28|        7|South Carolina|   Columbia|    Free Weight Bars|  Exercise & Fitness|105.20621383778517|
|2011|   10|        18| 